In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Layout, widgets, IntSlider, FloatSlider, Dropdown, IntText, Output
import scipy.signal
from math import log2
import random


# The function grayCode accepts a number n and returns an array g containing
# the Gray coding for numbers with n bits.
# This is done by recursively calling the function gray_code_recurse
def grayCode(n):
    def gray_code_recurse (g,n):
        k=len(g)
        if n<=0:
            return
        else:
            for i in range (k-1,-1,-1):
                char='1'+g[i]
                g.append(char)
            for i in range (k-1,-1,-1):
                g[i]='0'+g[i]

            gray_code_recurse (g,n-1)

    g=['0','1']
    gray_code_recurse(g,n-1)
    return g


# The function naturalBinaryCoding accepts a number n and returns an array
# with binary numbers with n bits (or equivalently up to the number 2**n, which results
# by left shifting 1 by n bits)
def naturalBinaryCoding(n):
    binary_levels = []
    for i in range(1 << n):
        binary_levels.append('{:0{}b}'.format(i, n))
    return binary_levels


# The function generateRandomBits generates n_bits random binary digits
def generateRandomBits(n_bits):
    bitstream = []
    for i in range(n_bits):
        random_bit = random.randint(0, 1)
        bitstream.append(random_bit)
    return bitstream


# The function createLevels divides the range [-A, A] into L-1 equal intervals,
# so that it always contains the endpoints of the range
def createLevels(A, L):
    y = []
    step = (A - (-A)) / (L - 1)
    for i in range(L):
        y.append(-A + i * step)
    return y


# The function createSymbols takes an array with bits as an argument and
# groups neighboring digits into symbols with length k
def createSymbols(k, bitstream):
    n_bits = len(bitstream)
    symbols = []
    for i in range(0, n_bits - k + 1, k):
        symbol = ""
        for j in range(k):
            symbol += str(bitstream[i+j])
        symbols.append(symbol)
    return symbols


# The function rootRaisedCosine creates a root raised cosine pulse
# with a roll-off coefficient and order determined by the sampling rate (nsample)
# and the delay it will introduce (delay)
def rootRaisedCosine(nsamp, roll_off, delay):
    F0 = 0.5 / nsamp
    Fd = 1
    Fs = Fd * nsamp
    Td = 1 / Fd
    Ts = 1 / Fs
    F1 = F0 * (1 - roll_off)
    F2 = F0 * (1 + roll_off)
    filter_order = 2 * nsamp * delay

    t = np.arange(0, filter_order, Td)
    h = []
    for i in range(len(t)):
        t_shifted = t[i] - filter_order / 2
        if t_shifted == 0:
            h.append(np.sqrt(2 * F0) *(1 + roll_off * ((4 / np.pi) - 1)))
        elif t_shifted == 1 / 8 / roll_off / F0 or t_shifted == - 1 / 8 / roll_off / F0 :
            h.append((roll_off * np.sqrt(F0)) * ((1 + 2 / np.pi) * np.sin(np.pi / 4 / roll_off) + (1 - 2 / np.pi) * np.cos(np.pi / 4 / roll_off)))
        else:
            factor1 = np.sqrt(2 * F0) / (1 - 64 * roll_off* roll_off * F0 * F0 * t_shifted * t_shifted)
            factor2 = np.sin(2 * np.pi * F1 * t_shifted) / (2 * np.pi * F0 * t_shifted)
            factor3 = (4 * roll_off / np.pi) * np.cos(2 * np.pi * F2 * t_shifted) 
            h.append(factor1 * (factor2 + factor3))
    
    return t,h


# The function upSample increases the number of samples of a signal signal by adding nsamp-1
# zeros after each sample of the signal
def upSample(signal, nsamp):
    upSampled = []
    for i in range(len(signal) * nsamp):
        if i % nsamp == 0:
            upSampled.append(signal[i // nsamp])
        else:
            upSampled.append(0)
    return upSampled


# The function downSample reduces the sampling frequency of a signal signal by a factor of nsamp
# by keeping only the samples that are multiples of nsamp (0, nsamp, 2*nsamp, ...)
def downSample(signal, nsamp):
    downSampled = []
    for i in range(0, len(signal), nsamp):
        downSampled.append(signal[i])
        
    return downSampled


# Adds white Gaussian noise with mean value μ (mu) and variance σ^2 (sigma)
def generateAWGN(signal, mu, sigma):
    noise = sigma * np.random.randn(len(signal)) + mu
    return noise



# Output widget for dynamic updates
output = Output()

# Global variables for filter and parameters
global filt, g_delay, g_nsamp

# Function for interactive signal processing
def interactive_signal_processing(n_bits, L, roll_off, nsamp, delay):
    # Set global variables for filter and nsamp/delay values
    global filt, g_delay, g_nsamp
    g_delay = delay
    g_nsamp = nsamp

    # Generate random bitstream
    bitstream = generateRandomBits(n_bits)

    # Determine ASK amplitude and number of bits per symbol
    A = L - 1
    k = int(log2(L))

    # Generate signal levels and symbols using Gray encoding
    y_levels = createLevels(A, L)
    symbols = createSymbols(k, bitstream)
    gray_encoding = grayCode(k)

    # Convert symbols to corresponding Gray code levels
    x_gray = [y_levels[gray_encoding.index(symbol)] for symbol in symbols]

    # Generate root-raised cosine filter and apply it to the signal
    t_filter, filt = rootRaisedCosine(nsamp, roll_off, delay)
    y = upSample(x_gray, nsamp)
    y_transmitted = scipy.signal.convolve(y, filt)
    y_received = scipy.signal.convolve(y_transmitted, filt)

    # Downsample received signal and remove delay effects
    y_final = downSample(y_received, nsamp)
    y_final = y_final[2 * delay: len(y_final) - 2 * delay]

    # Plot the results
    with output:
        output.clear_output(wait=True)  # Clear previous output
        fig = plt.figure(figsize=(20, 12))  # Create a figure object

        # First subplot - Root Raised Cosine Filter
        ax1 = fig.add_subplot(2, 2, 1)
        ax1.plot(t_filter, filt)
        ax1.set_title('Root Raised Cosine Filter')
        ax1.set_xlabel('Time')
        ax1.set_ylabel('Amplitude')
        ax1.grid(True)

        # Second subplot - Continuous signal with stems
        ax2 = fig.add_subplot(2, 2, 2)
        t_continuous = np.arange(0, len(y_transmitted[:10 * nsamp]))  # Time vector for continuous signal
        ax2.plot(t_continuous, y_transmitted[:10 * nsamp], label='Filtered Signal')

        # Stem positions for original symbols
        t_symbols = np.arange(0, 10 * nsamp, nsamp)  # Stem positions every nsamp samples
        y_stems = y_transmitted[t_symbols]  # Corresponding y-values for stems
        ax2.stem(t_symbols, y_stems, linefmt='C1-', markerfmt='C1o', basefmt=" ", label='Original Symbols')
        ax2.set_title('Signal Visualization')
        ax2.set_xlabel('Time')
        ax2.set_ylabel('Amplitude')
        ax2.legend()
        ax2.grid(True)

        # Third subplot - Signal Visualization
        ax3 = fig.add_subplot(2, 2, 3)
        t = np.arange(0, len(y[:10]))
        ax3.plot(t, y_final[:10])
        ax3.stem(t, x_gray[:10])
        ax3.set_title('Signal Visualization')
        ax3.legend(['Received','Transmitted'])
        ax3.set_xlabel('Time')
        ax3.set_ylabel('Amplitude')
        ax3.grid(True)

        # Fourth subplot - Power Spectral Density of the Received Signal
        ax4 = fig.add_subplot(2, 2, 4)
        f, Pxx_den = scipy.signal.welch(y_received, window='hamming', nperseg=8192)
        Pxx_den = 10 * np.log10(Pxx_den)
        ax4.plot(f, Pxx_den)
        ax4.set_title('Power Spectral Density of the Received Signal')
        ax4.set_xlabel('Normalized Frequency')
        ax4.set_ylabel('Power/Frequency [dB]')
        ax4.grid(True)

        plt.tight_layout()  # Adjust layout
        plt.show()

# Widget setup for interactive sliders and dropdowns
n_bits_slider = IntSlider(min=10000, max=100000, step=10000, value=10000, description='Bitstream Length', layout=Layout(width='100%'), continuous_update=False)
roll_off_slider = FloatSlider(min=0.1, max=1.0, step=0.1, value=0.4, description='Roll-off Factor', layout=Layout(width='100%'), continuous_update=False)
L_dropdown = Dropdown(options=[2**i for i in range(1, 6)], value=2, description='ASK Levels', continuous_update=False)
nsamp_slider = IntSlider(min=10, max=40, step=1, value=20, description='nsamp', layout=Layout(width='100%'), continuous_update=False)
delay_slider = IntSlider(min=1, max=10, step=1, value=5, description='Group Delay', layout=Layout(width='100%'), continuous_update=False)

# Display the filter order based on nsamp and delay
filter_order_display = IntText(value=2 * nsamp_slider.value * delay_slider.value, description='Filter Order:', disabled=True)

# Grouping inputs into a VBox
inputs = widgets.VBox([n_bits_slider, roll_off_slider, nsamp_slider, delay_slider, filter_order_display, L_dropdown])

# Function to update filter order based on nsamp and delay values
def update_filter_order(*args):
    filter_order = 2 * nsamp_slider.value * delay_slider.value
    filter_order_display.value = filter_order

# Observers to update filter order when nsamp or delay changes
nsamp_slider.observe(update_filter_order, 'value')
delay_slider.observe(update_filter_order, 'value')

# Call interactive signal processing function when parameters change
def on_change(*args):
    interactive_signal_processing(n_bits_slider.value, L_dropdown.value, roll_off_slider.value, nsamp_slider.value, delay_slider.value)

# Observe changes in the widgets and trigger signal processing
n_bits_slider.observe(on_change, 'value')
roll_off_slider.observe(on_change, 'value')
L_dropdown.observe(on_change, 'value')
nsamp_slider.observe(on_change, 'value')
delay_slider.observe(on_change, 'value')

# Display the UI and output area
display(inputs, output)

# Run the initial signal processing with default values
interactive_signal_processing(n_bits_slider.value, L_dropdown.value, roll_off_slider.value, nsamp_slider.value, delay_slider.value)


Output()